In [2]:
import numpy as np
import os
import os.path as osp
import os
import cv2
import json
import pickle
import imageio

In [3]:
base_path = '/group/wanbo/flickr30k_images/flickr30k_anno'
top10 = json.load(open(osp.join(base_path, 'top10_boxes_new.json')))
precompute_bbox = json.load(open(osp.join(base_path, 'precompute_proposals_nms_1e4.json')))
sentence_root = '{}/sent_anno.json'.format(base_path)
bbox_root = '{}/box_anno.json'.format(base_path)
sent_anno = json.load(open(sentence_root, 'r'))
bbox_anno = json.load(open(bbox_root, 'r'))

img_root_path = '/root/PycharmProjects/VisualGroundingAnalysis/datasets/flickr30k_images/{}.jpg'

In [4]:
def get_gt_boxes(bbox_anno, img_id):
    box_anno = bbox_anno[img_id]
    gt_boxes = []
    box_ids = []
    num_multiple_boxes = 0
    num_multiple_boxes_list = []
    for k, v in box_anno['boxes'].items():
        box_ids.append(k)
        if len(v) == 1:
            gt_boxes.append(v[0])
            num_multiple_boxes_list.append(0)
        else:
            # when a phrase respond to multiple regions, we take the union of them as paper given
            v = np.array(v)
            box = [v[:, 0].min(), v[:, 1].min(), v[:, 2].max(), v[:, 3].max()]
            gt_boxes.append(box)
            num_multiple_boxes += 1
            num_multiple_boxes_list.append(1)
    gt_boxes = np.array(gt_boxes)
    return box_ids, gt_boxes, num_multiple_boxes, num_multiple_boxes_list

def bbox_overlaps(anchors, gt_boxes):
    """
    anchors: (N, 4) ndarray of float
    gt_boxes: (K, 4) ndarray of float
    all numpy
    overlaps: (N, K) ndarray of overlap between boxes and query_boxes
    """
    
    anchors = torch.FloatTensor(anchors)
    gt_boxes=torch.FloatTensor(gt_boxes)
    N = anchors.size(0)
    K = gt_boxes.size(0)

    gt_boxes_area = ((gt_boxes[:,2] - gt_boxes[:,0] + 1) *
                (gt_boxes[:,3] - gt_boxes[:,1] + 1)).view(1, K)

    anchors_area = ((anchors[:,2] - anchors[:,0] + 1) *
                (anchors[:,3] - anchors[:,1] + 1)).view(N, 1)

    boxes = anchors.view(N, 1, 4).expand(N, K, 4)
    query_boxes = gt_boxes.view(1, K, 4).expand(N, K, 4)

    iw = (torch.min(boxes[:,:,2], query_boxes[:,:,2]) -
        torch.max(boxes[:,:,0], query_boxes[:,:,0]) + 1)
    iw[iw < 0] = 0

    ih = (torch.min(boxes[:,:,3], query_boxes[:,:,3]) -
        torch.max(boxes[:,:,1], query_boxes[:,:,1]) + 1)
    ih[ih < 0] = 0

    ua = anchors_area + gt_boxes_area - (iw * ih)
    overlaps = iw * ih / ua
    
    overlaps = overlaps.numpy()

    return overlaps

def clip_boxes(bbox, w, h):
    
    bbox[:, 0]  = np.maximum(bbox[:, 0], 0)
    bbox[:, 1]  = np.maximum(bbox[:, 1], 0)
    bbox[:, 2]  = np.minimum(bbox[:, 2], w)
    bbox[:, 3]  = np.minimum(bbox[:, 3], h)
    
    return bbox

def bbox_plot(bbox, img, color):
    
    pt1 = (bbox[0], bbox[1])
    pt2 = (bbox[2], bbox[3])
    img = cv2.rectangle(img, pt1, pt2, color, 2)
    return img

def remove_out_ids(pred_topN_id, num_boxes):

    num_phrase = pred_topN_id.shape[0]/10
    pred_topN_id = pred_topN_id.reshape(int(num_phrase), 10)
    remove_inds = (np.arange(num_phrase) * num_boxes)[:, None]
    
    remove_inds = remove_inds.astype(np.int32)
    pred_topN_id -= remove_inds
    pred_topN_id = pred_topN_id.reshape(-1).tolist()
    return pred_topN_id

In [7]:

img_ids = list(top10.keys())


img_save_path = '/p300/PycharmProjects/VisualGrounding/BoundingBoxesImg'


for img_id in img_ids:
    
    img = imageio.imread(img_root_path.format(img_id))
    h, w, c = img.shape
    
    precompute_bbox_img = np.array(precompute_bbox[img_id]).astype(np.int32)
    precompute_bbox_img = clip_boxes(precompute_bbox_img, w, h)
    num_precompute_box = precompute_bbox_img.shape[0]
    
    bbox_id, gt_boxes, num_multiple_boxes, num_multiple_boxes_list = get_gt_boxes(bbox_anno, img_id)
    img_global = img.copy()
    
    save_path = osp.join(img_save_path, img_id)
    if not osp.exists(save_path):
        os.makedirs(save_path)
    
    for pre_id in range(20):
        pre_bbox = precompute_bbox_img[pre_id]
        
        img_global = bbox_plot(pre_bbox, img_global, (255,192,0))
    
    imageio.imsave(osp.join(save_path, 'img_global.png'), img_global)
    print('save global image done')
    
    sent_ids = list(top10[img_id].keys())
    for sent_id in sent_ids:
        
        top_sent = top10[img_id][sent_id]
        sent = sent_anno[img_id][int(sent_id)]['sentence']
        phrases = sent_anno[img_id][int(sent_id)]['phrases']
        
        ## remove the phr not in box ids
        phrases_remain = []
        phrases_ids = []
        
        for phr in phrases:
            phr_id = phr['phrase_id']
            phrase = phr['phrase']
            if phr_id in bbox_id:
                phrases_remain.append(phrase)
                phrases_ids.append(phr_id)
        
                
        num_phrase = len(phrases_remain)
        num_pre_bbox = precompute_bbox_img

        top_sent_box_id = top_sent['box_id']
        
        for idx, top10_bbox_id in enumerate(top_sent_box_id):
            
            top10_bbox_id = np.array(top10_bbox_id) - idx*num_precompute_box
            top10_bbox = precompute_bbox_img[top10_bbox_id.astype(np.int32)]
            img_top = img.copy()
            
            bbox_gt = gt_boxes[bbox_id.index(phrases_ids[idx])]
            img_top = bbox_plot(bbox_gt, img_top, (255, 0, 0))
            for top_id in range(10):
                top_box = top10_bbox[top_id]
                img_top = bbox_plot(top_box, img_top, (255, 192, 0))
            cv2.putText(img_top, phrases_remain[idx], (10,30), cv2.FONT_HERSHEY_COMPLEX, 1, (255,192,0), 1)
            
            imageio.imsave(osp.join(save_path, 'img_{}_{}.png'.format(sent_id, idx)), img_top)
            print('save one sent img done')
    
    print(img_id, 'done')

                
                
            
            
            
            
            
        
        
        
               






save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
6540658813 done
save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
952352291 done
save global image done
save one sent img done
save one sent img done
save one sent img done
sav

save one sent img done
save one sent img done
6074238868 done
save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
4183630653 done
save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
1889398867 done
save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
96113759 

save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
1717077532 done
save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
3928410320 done
save global image done
save one sent img done
save one sent img done
sa

save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
3339263085 done
save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
2909081008 done
save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
3178454823 done
save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one 

save one sent img done
save one sent img done
save one sent img done
7453903222 done
save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
3249787980 done
save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
416825249 done
save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
221421001 done
save global image done
save one sent img 

...

save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
3126867551 done
save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
1279461814 done
save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
sa

save one sent img done
save one sent img done
save one sent img done
save one sent img done
363847768 done
save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
2671560649 done
save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
13652901 done
save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sen

save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
3239142107 done
save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
7228557720 done
save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
sa

save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
4623271967 done
save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
2822290399 done
save global image done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
save one sent img done
sa

KeyboardInterrupt: 